In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def load_dict_npy_robust(path: str):
    """Wczytuje słownik z .npy zapisanym przez np.save(..., allow_pickle=True)
    próbując różnych kodowań (latin1/bytes) typowych dla starych pickli."""
    for enc in (None, 'latin1', 'bytes'):
        try:
            if enc is None:
                arr = np.load(path, allow_pickle=True)
            else:
                arr = np.load(path, allow_pickle=True, encoding=enc)
            if isinstance(arr, dict):
                return arr
            # 0-wymiarowa tablica z obiektem (typowe)
            return arr.item()
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Nie udało się wczytać {path} żadnym kodowaniem. Ostatni błąd: {last_err}")


step = 20
dic_node = load_dict_npy_robust("dic_node.npy")
dic_disp = load_dict_npy_robust(f"{step}_dic_displacement.npy")

node_ids = sorted(dic_node.keys())
coords = np.array([dic_node[i] for i in node_ids], dtype=float)   # (N,2)
disp   = np.array([dic_disp[i] for i in node_ids], dtype=float)   # (N,2)

deformed = coords + disp
umag = np.linalg.norm(disp, axis=1)

outdir = Path("plots_step20"); outdir.mkdir(exist_ok=True)

# quiver (rzadsze strzałki)
N = len(coords)
stride = max(1, N // 2000)
sel = slice(0, None, stride)

plt.figure(figsize=(7,5.5))
plt.quiver(coords[sel,0], coords[sel,1], disp[sel,0], disp[sel,1], angles='xy', scale_units='xy', scale=1)
plt.gca().set_aspect("equal", adjustable="box")
plt.title(f"Displacement vectors (step {step})  n={N}, stride={stride}")
plt.xlabel("x"); plt.ylabel("y")
plt.tight_layout(); plt.savefig(outdir / f"quiver_step{step}.png", dpi=160); plt.close()

# |u|
plt.figure(figsize=(7,5.5))
sc = plt.scatter(coords[:,0], coords[:,1], c=umag, s=8)
plt.colorbar(sc, label="|u|")
plt.gca().set_aspect("equal", adjustable="box")
plt.title(f"Displacement magnitude |u| (step {step})")
plt.xlabel("x"); plt.ylabel("y")
plt.tight_layout(); plt.savefig(outdir / f"umag_step{step}.png", dpi=160); plt.close()

# undeformed vs deformed
plt.figure(figsize=(7,5.5))
plt.scatter(coords[:,0], coords[:,1], s=5, label="undeformed", alpha=0.35)
plt.scatter(deformed[:,0], deformed[:,1], s=5, label="deformed", alpha=0.75)
plt.legend(); plt.gca().set_aspect("equal", adjustable="box")
plt.title(f"Undeformed vs Deformed nodes (step {step})")
plt.xlabel("x"); plt.ylabel("y")
plt.tight_layout(); plt.savefig(outdir / f"overlay_step{step}.png", dpi=160); plt.close()

print("Wygenerowano wykresy w:", outdir)


Wygenerowano wykresy w: plots_step20
